In [4]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional

# Data Structures
class Task:
    def __init__(self, name: str, deadline: datetime, priority: int, effort: float):
        self.name = name
        self.deadline = deadline
        self.priority = priority
        self.effort = effort
        self.score = 0.0

    def __repr__(self):
        return f"Task({self.name}, deadline={self.deadline.date()}, priority={self.priority}, effort={self.effort}h, score={self.score:.2f})"


# Task Input
def validateTasks(tasks: List[Task]) -> List[Task]:
    valid = []
    for t in tasks:
        if t.effort <= 0:
            print(f"Warning: Task '{t.name}' has non-positive effort, skipping.")
            continue
        if t.deadline < datetime.now():
            print(f"Warning: Task '{t.name}' deadline already passed, skipping.")
            continue
        valid.append(t)
    return valid
def calculatePriorityScore(task: Task, currTime: datetime) -> float:

    daysLeft = max((task.deadline - currTime).days, 0.1)
    pWeight = 0.5
    uWeight = 0.4
    eWeight = 0.1

    pComp = task.priority
    uComp = 1 / daysLeft
    eComp = 1 / task.effort

    score = (pComp * pWeight +
             uComp * uWeight +
             eComp * eWeight)
    return score
# Scheduling Engine
def scheduleTasks(tasks: List[Task], startDate: datetime, days: int = 7) -> Dict[datetime, List[Task]]:

    currTime = startDate
    for task in tasks:
        task.score = calculatePriorityScore(task, currTime)
    tasks = sorted(tasks, key=lambda t: t.score, reverse=True)

    schedule = {}
    dailyHours = 8.0
    slotHours = 4.0

    for i in range(days):
        day = startDate + timedelta(days=i)
        schedule[day.date()] = {'morning': [], 'afternoon': [], 'morningHours': 0, 'afternoonHours': 0}

    remainingTasks = list(tasks)

    for task in remainingTasks:
        taskToSchedule = task

        for i in range(days):
            day = startDate + timedelta(days=i)
            dayKey = day.date()

            morningCapacity = slotHours - schedule[dayKey]['morningHours']
            afternoonCapacity = slotHours - schedule[dayKey]['afternoonHours']

            if taskToSchedule.score > 0.7 and morningCapacity > 0:
                if taskToSchedule.effort <= morningCapacity:
                    schedule[dayKey]['morning'].append(taskToSchedule)
                    schedule[dayKey]['morningHours'] += taskToSchedule.effort
                    taskToSchedule.effort = 0
                    break
                else:

                    scheduledEffort = morningCapacity
                    partTask = Task(f"{taskToSchedule.name} (part)", taskToSchedule.deadline, taskToSchedule.priority, scheduledEffort)
                    schedule[dayKey]['morning'].append(partTask)
                    schedule[dayKey]['morningHours'] += scheduledEffort
                    taskToSchedule.effort -= scheduledEffort

            if taskToSchedule.effort > 0 and afternoonCapacity > 0:
                if taskToSchedule.effort <= afternoonCapacity:
                    schedule[dayKey]['afternoon'].append(taskToSchedule)
                    schedule[dayKey]['afternoonHours'] += taskToSchedule.effort
                    taskToSchedule.effort = 0
                    break
                else:
                    scheduledEffort = afternoonCapacity
                    partTask = Task(f"{taskToSchedule.name} (part)", taskToSchedule.deadline, taskToSchedule.priority, scheduledEffort)
                    schedule[dayKey]['afternoon'].append(partTask)
                    schedule[dayKey]['afternoonHours'] += scheduledEffort
                    taskToSchedule.effort -= scheduledEffort


        if taskToSchedule.effort > 0:
             print(f"Warning: Could not schedule all of task '{taskToSchedule.name}'. Remaining effort: {taskToSchedule.effort}h")

    return schedule

def printSchedule(schedule: Dict[datetime, Dict]):
    print("\nOptimized Task Schedule:\n" + "="*25)
    for day, slots in schedule.items():
        print(f"\nDate: {day.strftime('%Y-%m-%d')}")
        print("  Morning:")
        if slots['morning']:
            for t in slots['morning']:
                print(f"    - {t.name} (Effort: {t.effort}h, Priority: {t.priority}, Deadline: {t.deadline.date()})")
        else:
            print("    (No tasks)")
        print("  Afternoon:")
        if slots['afternoon']:
            for t in slots['afternoon']:
                print(f"    - {t.name} (Effort: {t.effort}h, Priority: {t.priority}, Deadline: {t.deadline.date()})")
        else:
            print("    (No tasks)")
if __name__ == "__main__":
    now = datetime.now()

    tasks = [
        Task("Finish report", now + timedelta(days=2), priority=5, effort=3),
        Task("cold emailing", now + timedelta(days=1), priority=4, effort=1),
        Task("Prepare presentation", now + timedelta(days=5), priority=3, effort=5),
        Task("Code review", now + timedelta(days=3), priority=2, effort=2),
        Task("complete hackathon project", now + timedelta(days=7), priority=1, effort=1.5),
        Task("Plan Meeting", now + timedelta(days=1), priority=5, effort=4),
        Task("Write documentation", now + timedelta(days=6), priority=3, effort=3),
    ]

    validtasks = validateTasks(tasks)
    schedule = scheduleTasks(validtasks, startDate=now, days=7)
    printSchedule(schedule)


Optimized Task Schedule:

Date: 2025-07-08
  Morning:
    - Plan Meeting (Effort: 0h, Priority: 5, Deadline: 2025-07-09)
  Afternoon:
    - Finish report (Effort: 0h, Priority: 5, Deadline: 2025-07-10)
    - cold emailing (Effort: 0h, Priority: 4, Deadline: 2025-07-09)

Date: 2025-07-09
  Morning:
    - Prepare presentation (part) (Effort: 4.0h, Priority: 3, Deadline: 2025-07-13)
  Afternoon:
    - Prepare presentation (Effort: 0h, Priority: 3, Deadline: 2025-07-13)
    - Write documentation (Effort: 0h, Priority: 3, Deadline: 2025-07-14)

Date: 2025-07-10
  Morning:
    - Code review (Effort: 0h, Priority: 2, Deadline: 2025-07-11)
  Afternoon:
    - complete hackathon project (Effort: 0h, Priority: 1, Deadline: 2025-07-15)

Date: 2025-07-11
  Morning:
    (No tasks)
  Afternoon:
    (No tasks)

Date: 2025-07-12
  Morning:
    (No tasks)
  Afternoon:
    (No tasks)

Date: 2025-07-13
  Morning:
    (No tasks)
  Afternoon:
    (No tasks)

Date: 2025-07-14
  Morning:
    (No tasks)
  Aft